# Import Dataset

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
### 1. Link notebook with google drive and access data from your personal Gdrive
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')

### 2.Set the data path for dataset and model location (ex: model_loc = "/content/gdrive/My Drive/Dataset/")
dataset_dir = "/content/gdrive/My Drive/BioinfoModelingAndSimulation/"
model_loc = "/content/gdrive/My Drive/BioinfoModelingAndSimulation/"

print(os.listdir(dataset_dir))
mrna_data = pd.read_csv(dataset_dir+'BRCA_mRNA.csv', index_col=0)
miRNA_data = pd.read_csv(dataset_dir+'BRCA_miRNA.csv', index_col=0)
DNA_Methylation_data = pd.read_csv(dataset_dir+'BRCA_DNA_Methylation.csv', index_col=0)
copy_number_variation_data = pd.read_csv(dataset_dir+'BRCA_Copy Number Variation.csv', index_col=0)
multi_omics_data = pd.read_csv(dataset_dir+'BRCA_Multi_Omics.csv', index_col=0)
brca_Label = pd.read_csv(dataset_dir+'BRCA_label.csv')

Mounted at /content/gdrive
['heart.csv', 'Heart_Disease_NN.ipynb', 'BRCA_miRNA.csv', 'BRCA_mRNA.csv', 'BRCA_DNA_Methylation.csv', 'BRCA_label.csv', 'BRCA_Copy Number Variation.csv', 'X_test_rfe_mrna.pkl', 'X_train_rfe_mrna.pkl', 'X_train_rfe_miRNA.pkl', 'X_test_rfe_miRNA.pkl', 'X_test_rfe_dna.pkl', 'X_train_rfe_dna.pkl', 'BRCA_Multi_Omics.csv', 'X_test_rfe_copy_number_variation.pkl', 'X_train_rfe_multi_omics.pkl', 'X_train_rfe_copy_number_variation.pkl', 'X_test_rfe_multi_omics.pkl', 'SDAE_WITHOUT_SMOTE.ipynb']


#Data Preprocessing & EDA


## BRCA_Copy Number Variation

In [ ]:
copy_number_variation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19568 entries, A1BG to pk
Columns: 671 entries, TCGA.3C.AAAU.01 to TCGA.Z7.A8R6.01
dtypes: float64(671)
memory usage: 100.3+ MB


In [ ]:
copy_number_variation_data.head(10)

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,0.495252,1.028904,0.003059,-0.258705,-0.083232,-0.486242,-0.111192,-0.102997,0.687116,-0.213873,...,-0.493956,2.097171,-0.594226,0.478862,-1.755051,-0.835261,-0.148311,0.557439,-0.234602,-0.838154
A1CF,-0.938762,-1.575977,0.405308,0.096910,0.060782,2.050968,0.154506,0.018894,1.131535,0.145081,...,-0.299975,6.501006,0.255559,-0.184784,0.371274,0.483323,0.061306,0.627836,0.070730,-0.463861
A2M,0.750838,0.010864,0.103201,-0.169130,-2.090765,-0.216362,-0.062751,-0.056368,0.857217,-0.016369,...,1.139760,-1.938856,0.036395,-0.308274,0.118520,0.266599,-0.089133,0.344469,0.226601,-0.590817
A2ML1,0.748593,0.005894,0.098571,-0.174763,-2.103474,-0.222169,-0.067992,-0.061585,0.855364,-0.021439,...,1.138948,-1.951005,0.031519,-0.314419,0.113946,0.262571,-0.094471,0.340728,0.222425,-0.598003
A3GALT2,1.093743,-1.575680,-0.706498,0.199712,0.376570,-2.334753,0.400443,0.225047,1.226264,0.390699,...,-2.541330,-1.587373,0.371698,0.096424,-1.494803,-2.569588,0.046241,-1.244865,-0.252418,-0.264599
A4GALT,-0.876388,-0.967673,-1.643561,0.570857,0.631333,2.749902,0.620683,0.497449,0.387146,-0.143066,...,-1.655732,0.592157,-1.734085,0.276083,0.805155,-1.733704,0.592537,-0.629158,0.087808,0.082483
A4GNT,1.109460,1.481901,-0.661117,-0.422884,-0.350660,-0.173333,-0.273585,-0.151234,-0.508045,-0.384077,...,1.316971,2.246187,-1.068054,-0.615303,1.713128,-0.412105,-0.391623,-0.897733,-1.021162,-0.788319
AAAS,-1.966983,-0.040373,-1.342185,-0.156210,-0.101592,-0.175416,0.005842,0.145686,1.277645,-0.019366,...,1.799210,-2.374512,0.098871,-0.207226,0.242317,0.359954,-0.094390,0.600030,0.488395,-0.545733
AACS,-1.676420,-0.027748,1.469986,-0.017618,0.100904,0.011759,0.084190,-1.943348,-1.716941,0.046708,...,1.439089,-1.900802,0.146490,-2.367799,0.293883,0.516238,0.009733,0.666163,0.491419,-0.372678
AADAC,1.090124,1.213108,-0.571499,-0.402285,-0.312965,-0.110984,-0.226338,-0.146892,-0.444027,-0.340793,...,-0.534694,0.613451,-0.853823,-0.533347,1.399827,-0.364133,-0.347077,-0.768542,-0.882100,-0.628054


In [ ]:
copy_number_variation_data.iloc[0:,1:]

,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,TCGA.A1.A0SH.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,1.028904,0.003059,-0.258705,-0.083232,-0.486242,-0.111192,-0.102997,0.687116,-0.213873,-0.005619,...,-0.493956,2.097171,-0.594226,0.478862,-1.755051,-0.835261,-0.148311,0.557439,-0.234602,-0.838154
A1CF,-1.575977,0.405308,0.096910,0.060782,2.050968,0.154506,0.018894,1.131535,0.145081,0.042456,...,-0.299975,6.501006,0.255559,-0.184784,0.371274,0.483323,0.061306,0.627836,0.070730,-0.463861
A2M,0.010864,0.103201,-0.169130,-2.090765,-0.216362,-0.062751,-0.056368,0.857217,-0.016369,-0.034667,...,1.139760,-1.938856,0.036395,-0.308274,0.118520,0.266599,-0.089133,0.344469,0.226601,-0.590817
A2ML1,0.005894,0.098571,-0.174763,-2.103474,-0.222169,-0.067992,-0.061585,0.855364,-0.021439,-0.039804,...,1.138948,-1.951005,0.031519,-0.314419,0.113946,0.262571,-0.094471,0.340728,0.222425,-0.598003
A3GALT2,-1.575680,-0.706498,0.199712,0.376570,-2.334753,0.400443,0.225047,1.226264,0.390699,0.612866,...,-2.541330,-1.587373,0.371698,0.096424,-1.494803,-2.569588,0.046241,-1.244865,-0.252418,-0.264599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,-1.793233,-0.872324,0.058733,0.242915,-2.475162,0.263210,0.085117,1.176508,0.207398,0.322067,...,-0.169592,0.192176,0.237841,-0.048833,-1.672982,-2.900353,-0.073695,-0.277665,-0.435463,-0.424808
ZYX,1.446230,0.039191,-0.257130,-0.185487,-1.970238,-0.112381,-0.106533,0.704939,-0.262491,0.373527,...,-0.542729,-0.489606,-0.134313,0.310169,-0.002235,1.155757,-0.286372,0.253634,-0.327799,-0.589030
ZZEF1,-0.790000,-1.421040,-1.377441,-0.999581,1.646581,0.848410,-0.727278,-0.501251,0.866003,-0.575828,...,-1.375911,-0.695152,0.920311,-0.919650,-0.690180,0.438043,0.811313,-0.470272,0.723732,-2.171404
ZZZ3,-1.268621,-0.811368,0.098802,0.204322,-1.930651,0.307432,0.230822,-1.278740,0.290087,0.216849,...,-2.491497,-0.100674,0.289605,-0.009609,-1.555790,-2.902013,-0.051046,-0.247150,-0.350742,-0.339178


In [ ]:
# check is there any null values in the BRCA_Copy Number Variation dataset
copy_number_variation_data.isnull().any().any()

False

In [ ]:
copy_number_variation_data.shape

(19568, 671)

## BRCA_miRNA

In [ ]:
miRNA_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 368 entries, hsa.let.7a.1 to hsa.mir.99b
Columns: 671 entries, TCGA.3C.AAAU.01 to TCGA.Z7.A8R6.01
dtypes: float64(671)
memory usage: 1.9+ MB


In [ ]:
miRNA_data.head(10)

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
hsa.let.7a.1,0.068317,-0.301684,-0.150810,0.107831,0.395211,1.135212,1.813887,0.473642,-0.470612,0.791968,...,-3.038971,-0.474111,0.165775,0.706495,-0.086149,0.511958,1.225298,0.667662,-0.211878,0.474240
hsa.let.7a.2,0.068932,-0.318009,-0.122747,0.097594,0.412879,1.140183,1.790792,0.442224,-0.461217,0.841821,...,-3.037340,-0.477443,0.172506,0.706552,-0.074380,0.504240,1.219548,0.666012,-0.210525,0.488117
hsa.let.7a.3,0.073899,-0.301310,-0.126333,0.095545,0.418441,1.143605,1.822272,0.431312,-0.470734,0.805374,...,-3.017967,-0.474484,0.164726,0.727433,-0.102883,0.496009,1.210610,0.675998,-0.220135,0.481577
hsa.let.7b,0.524562,0.419859,-0.958939,0.615389,0.500594,1.159087,2.113821,0.812876,-0.129307,0.867411,...,-3.509653,-0.855498,-0.359797,0.997679,-0.342075,0.422382,0.050804,-0.102655,0.177655,-0.078426
hsa.let.7c,-1.656853,-0.715963,-0.971038,0.711952,0.426323,-1.058870,2.135373,1.031809,0.864211,0.071249,...,-2.454470,0.188937,-0.001077,1.302563,0.506728,1.028977,1.186182,1.534247,1.070328,-0.155655
hsa.let.7d,-0.038283,0.460975,0.866585,-0.454282,-1.545556,0.348033,1.844765,-0.453824,-1.070594,0.499824,...,0.391518,-0.452204,-0.049724,0.053945,0.627048,-0.254668,-1.238797,0.305716,-0.716526,1.384395
hsa.let.7e,0.501125,-1.999304,2.074809,0.227441,-0.952282,1.610267,-0.469603,-0.611333,-0.571769,0.505744,...,-0.503044,-0.048044,1.286869,0.297414,0.303064,0.528134,0.879213,1.276369,-0.598473,1.237552
hsa.let.7f.1,-2.390084,-0.659788,1.080746,-0.735552,-0.280966,-0.141723,0.837358,0.169462,-0.000266,0.859450,...,-1.216759,0.095809,0.834571,-0.058999,0.798165,0.757211,1.455206,1.253312,-0.229774,0.851405
hsa.let.7f.2,-2.406331,-0.651943,1.088164,-0.686623,-0.233746,-0.152147,0.798862,0.184119,0.041877,0.851887,...,-1.168059,0.082452,0.833022,-0.050336,0.818886,0.741367,1.460326,1.240593,-0.190906,0.851526
hsa.let.7g,-0.760042,-1.050266,0.523204,-1.015715,-0.818574,1.243074,2.111705,-0.511843,-1.315689,0.162780,...,0.156156,0.053724,0.091651,0.653836,1.183322,0.378499,0.921697,0.556762,-0.465549,0.409391


In [ ]:
# check is there any null values in the miRNA_data dataset
miRNA_data.isnull().any().any()

False

In [ ]:
miRNA_data.shape

(368, 671)

## BRCA_DNA_Methylation

In [ ]:
DNA_Methylation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19049 entries, A1BG to pk
Columns: 671 entries, TCGA.3C.AAAU.01 to TCGA.Z7.A8R6.01
dtypes: float64(671)
memory usage: 97.7+ MB


In [ ]:
# DNA_Methylation_dataset
DNA_Methylation_data.head(10)

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,-1.211396,0.974531,0.633283,0.354642,-1.939963,1.328608,0.946010,0.930281,-0.519615,0.554409,...,-0.760042,1.159665,0.333349,0.775567,-0.778884,0.517100,0.177384,-0.380611,0.811548,-0.819564
A1CF,-1.801239,-1.091976,-0.258825,0.344929,-0.018208,-1.007907,1.246215,0.360814,-0.426866,-1.357345,...,-0.582298,-2.142905,-0.306365,0.816486,-0.499731,0.824666,0.807188,-0.027147,0.644539,-2.004627
A2M,-0.696763,-1.597015,-0.873714,-1.080573,-0.788607,-0.583001,-0.654375,0.302439,-1.561816,-0.836623,...,-1.001059,-2.915892,0.635853,0.027964,-1.145160,-0.326970,0.569131,-2.613169,-0.507579,-0.339844
A2ML1,0.832338,0.758754,0.753608,0.548382,0.589277,0.870132,-1.878898,0.160075,0.066117,-0.338768,...,0.892903,-2.480427,0.319169,-0.098838,0.830086,0.617726,0.551416,0.101744,0.269641,-0.094397
A3GALT2,0.857098,0.450506,0.821330,0.497375,0.400632,1.183863,-3.316770,0.227579,0.051009,0.300039,...,1.161783,-1.863239,0.350236,-0.879849,-0.098983,0.524779,0.041604,0.537643,-0.322718,0.007108
A4GALT,-0.695053,1.570301,-0.369450,1.342464,1.553188,-0.876086,-1.168542,1.418889,-0.012446,-0.980884,...,-1.081537,1.713744,-0.986258,1.658525,-0.840527,0.887775,0.890293,-0.910491,0.886293,-1.066638
A4GNT,-0.258891,0.330579,-0.315881,-0.518794,-0.299152,0.562873,0.646204,0.812077,-0.205157,-2.506433,...,1.003214,-0.080964,0.514651,1.080107,-0.263919,0.553903,0.012863,-1.371328,0.555758,-1.140229
AAAS,0.601832,-0.912745,0.098281,-0.274796,0.391160,3.940350,-0.857682,0.537300,-0.532170,-0.666725,...,-0.023642,-0.410331,0.899429,-0.651502,0.026079,0.059472,-1.045639,-0.607679,-0.144756,-1.179954
AACS,-0.100803,-0.682832,-5.269406,-0.187178,-0.410440,-0.422690,-0.588374,0.343098,0.397713,0.234600,...,-0.225294,-0.520869,0.015193,0.462241,-0.235900,0.150010,0.102895,0.085117,-0.187391,0.503692
AADAC,-5.232067,-1.232277,-2.441139,-0.232070,-1.676137,-1.472467,0.997476,0.563539,0.714475,0.571410,...,0.008583,-1.804608,0.275197,0.574451,-1.478779,-0.999820,0.209390,-1.533629,-0.931151,0.063754


In [ ]:
# check is there any null values in the BRCA_DNA_Methylation dataset
DNA_Methylation_data.isnull().any().any()

False

In [ ]:
DNA_Methylation_data.shape

(19049, 671)

## BRCA_mRNA

In [ ]:
mrna_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18206 entries, A1BG to ZZZ3
Columns: 671 entries, TCGA.3C.AAAU.01 to TCGA.Z7.A8R6.01
dtypes: float64(671)
memory usage: 93.3+ MB


In [ ]:
mrna_data.head(10)

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,1.389927,1.522872,1.788271,-0.288543,-0.462680,-0.451169,-0.805973,-0.472368,0.112737,-0.397480,...,-1.828641,0.756934,1.095019,2.453552,-1.526821,0.108732,-0.219390,-0.112740,-0.920878,0.079873
A1BG.AS1,0.436758,1.815091,0.580557,-0.250194,-0.091285,-0.688598,-1.037605,0.658661,0.627872,0.426477,...,-2.123590,-0.556860,-1.271019,1.915429,-2.519037,-0.643504,0.082909,0.284536,-0.372892,0.539919
A2M,-0.876187,-0.508085,-0.258138,-0.030804,0.289970,-1.641876,1.534552,0.436717,0.476992,0.147622,...,-1.493460,-1.848483,-0.608446,0.584722,-0.267116,-1.271778,0.780906,0.292935,1.883640,-0.469512
A2M.AS1,1.607818,0.863019,-0.491954,-0.579638,0.572883,-0.836306,0.099614,-0.059346,-0.092982,-0.666693,...,0.851988,-2.180389,-1.614497,0.398519,-0.632821,-0.768806,0.200613,0.351478,0.015328,-1.309134
A2ML1,-0.415138,-0.406608,-0.455969,-0.464938,-0.438571,-0.479607,-0.444955,-0.539629,-0.408042,-0.457548,...,0.419145,-0.483924,-0.379297,-0.406793,-0.470376,-0.429807,-0.491028,0.421906,-0.456669,-0.394471
A2MP1,-0.374641,-0.361230,0.855983,-0.313092,-0.090844,-1.014920,1.384198,-0.508012,-0.264776,-0.343200,...,-0.814826,-0.889906,-0.577070,-0.209513,1.692166,-0.517498,1.236668,0.277833,2.279637,-0.730764
A4GALT,-1.727152,-0.964244,0.750127,0.480557,0.179004,0.420826,0.260066,0.058950,0.676890,1.261371,...,0.436093,-2.023100,1.798225,-0.706899,2.508851,-0.148959,0.397136,0.658981,0.319417,1.304757
AAAS,-0.562530,0.525889,-0.401461,-0.426938,0.685542,0.697997,0.127438,-0.117362,1.583352,0.208439,...,1.841616,-2.215116,1.540387,1.881914,1.241782,1.136479,-0.030975,0.359229,0.667426,0.433323
AACS,0.225676,-0.590749,1.077314,-0.739098,-0.623465,0.669477,1.339599,-0.443098,-1.331414,0.918643,...,1.194196,1.802331,0.876724,-1.193957,-0.788798,-1.740086,0.730476,-0.336013,-0.298045,-0.811775
AADAT,-1.337786,-0.724950,0.101880,-0.091341,-0.421059,1.067540,1.377738,-0.566328,-1.010971,-0.671109,...,0.511056,-1.219445,-0.981982,-0.682073,-0.389474,-0.562701,-0.449559,-0.483281,-0.376687,-0.699855


In [ ]:
# Check if there any null values in the BRCA_mRNA dataset
mrna_data.isnull().any().any()

False

## BRCA_label

In [ ]:
brca_Label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   671 non-null    object
dtypes: object(1)
memory usage: 5.4+ KB


In [ ]:
# BRCA_Copy Number Variation dataset
brca_Label.head(10)

,Label
0,LumA
1,Her2
2,LumB
3,LumA
4,LumA
5,LumB
6,Normal
7,LumA
8,LumA
9,LumA


In [ ]:
# check is there any null values in the BRCA_Copy Number Variation dataset
brca_Label.isnull().any().any()

False

In [ ]:
brca_Label.shape

(671, 1)

In [ ]:
label_value_counts = brca_Label['Label'].value_counts()

print(label_value_counts)

Label
LumA      353
LumB      132
Basal     113
Her2       42
Normal     31
Name: count, dtype: int64


In [ ]:
#replace label with numerical data
brca_Label['Label'].replace('Normal', 0 , inplace=True)
brca_Label['Label'].replace('LumA', 1 , inplace=True)
brca_Label['Label'].replace('LumB', 2 , inplace=True)
brca_Label['Label'].replace('Basal', 3 , inplace=True)
brca_Label['Label'].replace('Her2', 4 , inplace=True)

In [ ]:
label_value_counts = brca_Label['Label'].value_counts()

print(label_value_counts)

Label
1    353
2    132
3    113
4     42
0     31
Name: count, dtype: int64


## BRCA_Multi_Omics

In [ ]:
multi_omics_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57191 entries, A1BG to pk
Columns: 671 entries, TCGA.3C.AAAU.01 to TCGA.Z7.A8R6.01
dtypes: float64(671)
memory usage: 293.2+ MB


In [ ]:
multi_omics_data.head(10)

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
A1BG,0.495252,1.028904,0.003059,-0.258705,-0.083232,-0.486242,-0.111192,-0.102997,0.687116,-0.213873,...,-0.493956,2.097171,-0.594226,0.478862,-1.755051,-0.835261,-0.148311,0.557439,-0.234602,-0.838154
A1CF,-0.938762,-1.575977,0.405308,0.096910,0.060782,2.050968,0.154506,0.018894,1.131535,0.145081,...,-0.299975,6.501006,0.255559,-0.184784,0.371274,0.483323,0.061306,0.627836,0.070730,-0.463861
A2M,0.750838,0.010864,0.103201,-0.169130,-2.090765,-0.216362,-0.062751,-0.056368,0.857217,-0.016369,...,1.139760,-1.938856,0.036395,-0.308274,0.118520,0.266599,-0.089133,0.344469,0.226601,-0.590817
A2ML1,0.748593,0.005894,0.098571,-0.174763,-2.103474,-0.222169,-0.067992,-0.061585,0.855364,-0.021439,...,1.138948,-1.951005,0.031519,-0.314419,0.113946,0.262571,-0.094471,0.340728,0.222425,-0.598003
A3GALT2,1.093743,-1.575680,-0.706498,0.199712,0.376570,-2.334753,0.400443,0.225047,1.226264,0.390699,...,-2.541330,-1.587373,0.371698,0.096424,-1.494803,-2.569588,0.046241,-1.244865,-0.252418,-0.264599
A4GALT,-0.876388,-0.967673,-1.643561,0.570857,0.631333,2.749902,0.620683,0.497449,0.387146,-0.143066,...,-1.655732,0.592157,-1.734085,0.276083,0.805155,-1.733704,0.592537,-0.629158,0.087808,0.082483
A4GNT,1.109460,1.481901,-0.661117,-0.422884,-0.350660,-0.173333,-0.273585,-0.151234,-0.508045,-0.384077,...,1.316971,2.246187,-1.068054,-0.615303,1.713128,-0.412105,-0.391623,-0.897733,-1.021162,-0.788319
AAAS,-1.966983,-0.040373,-1.342185,-0.156210,-0.101592,-0.175416,0.005842,0.145686,1.277645,-0.019366,...,1.799210,-2.374512,0.098871,-0.207226,0.242317,0.359954,-0.094390,0.600030,0.488395,-0.545733
AACS,-1.676420,-0.027748,1.469986,-0.017618,0.100904,0.011759,0.084190,-1.943348,-1.716941,0.046708,...,1.439089,-1.900802,0.146490,-2.367799,0.293883,0.516238,0.009733,0.666163,0.491419,-0.372678
AADAC,1.090124,1.213108,-0.571499,-0.402285,-0.312965,-0.110984,-0.226338,-0.146892,-0.444027,-0.340793,...,-0.534694,0.613451,-0.853823,-0.533347,1.399827,-0.364133,-0.347077,-0.768542,-0.882100,-0.628054


In [ ]:
# Check if there any null values in the BRCA_mRNA dataset
multi_omics_data.isnull().any().any()

False

There is no missing values in all five dataset

# Data Preparation

Trasnpose the row and column

## BRCA_Copy Number Variation

In [ ]:
# Before transpose
copy_number_variation_data.shape

(19568, 671)

In [ ]:
# Transpose mrna_data
copy_number_variation_data = copy_number_variation_data.transpose()

In [ ]:
# After transpose
copy_number_variation_data.head(10)

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,0.495252,-0.938762,0.750838,0.748593,1.093743,-0.876388,1.109460,-1.966983,-1.676420,1.090124,...,-0.837399,2.020624,1.972719,1.053381,1.254989,1.267331,0.963246,-0.580418,1.715811,-0.864183
TCGA.3C.AALI.01,1.028904,-1.575977,0.010864,0.005894,-1.575680,-0.967673,1.481901,-0.040373,-0.027748,1.213108,...,-1.537275,4.002085,3.897638,1.589442,-1.781406,-1.793233,1.446230,-0.790000,-1.268621,0.345052
TCGA.3C.AALJ.01,0.003059,0.405308,0.103201,0.098571,-0.706498,-1.643561,-0.661117,-1.342185,1.469986,-0.571499,...,4.165544,-0.319977,-0.301093,-0.477515,-0.867769,-0.872324,0.039191,-1.421040,-0.811368,-1.357237
TCGA.3C.AALK.01,-0.258705,0.096910,-0.169130,-0.174763,0.199712,0.570857,-0.422884,-0.156210,-0.017618,-0.402285,...,0.080744,-0.316675,-0.297884,-0.267586,0.055935,0.058733,-0.257130,-1.377441,0.098802,0.042601
TCGA.5L.AAT0.01,-0.083232,0.060782,-2.090765,-2.103474,0.376570,0.631333,-0.350660,-0.101592,0.100904,-0.312965,...,0.045801,-0.307593,-0.289062,-0.200167,0.238663,0.242915,-0.185487,-0.999581,0.204322,0.078553
TCGA.5T.A9QA.01,-0.486242,2.050968,-0.216362,-0.222169,-2.334753,2.749902,-0.173333,-0.175416,0.011759,-0.110984,...,1.970711,-0.399235,-0.392526,-0.141518,-2.457950,-2.475162,-1.970238,1.646581,-1.930651,-6.994250
TCGA.A1.A0SB.01,-0.111192,0.154506,-0.062751,-0.067992,0.400443,0.620683,-0.273585,0.005842,0.084190,-0.226338,...,0.136450,-0.179623,-0.164744,-0.084514,0.258798,0.263210,-0.112381,0.848410,0.307432,0.189832
TCGA.A1.A0SE.01,-0.102997,0.018894,-0.056368,-0.061585,0.225047,0.497449,-0.151234,0.145686,-1.943348,-0.146892,...,0.005287,-0.121831,-0.108601,0.039909,0.082111,0.085117,-0.106533,-0.727278,0.230822,0.289698
TCGA.A1.A0SF.01,0.687116,1.131535,0.857217,0.855364,1.226264,0.387146,-0.508045,1.277645,-1.716941,-0.444027,...,1.072825,-0.210171,-0.194420,-0.356381,1.164884,1.176508,0.704939,-0.501251,-1.278740,-0.088652
TCGA.A1.A0SG.01,-0.213873,0.145081,-0.016369,-0.021439,0.390699,-0.143066,-0.384077,-0.019366,0.046708,-0.340793,...,0.127335,-0.254754,-0.237731,-0.196879,0.203426,0.207398,-0.262491,0.866003,0.290087,0.104803


In [ ]:
copy_number_variation_data.shape

(671, 19568)

In [ ]:
# Ensure the number of rows in data match with label
if copy_number_variation_data.shape[0] == brca_Label.shape[0]:
    # Add the new column to the transposed DataFrame
    copy_number_variation_data['Label'] = brca_Label['Label'].values
else:
    raise ValueError("The number of rows in must match the number of columns in the transposed df1.")

In [ ]:
copy_number_variation_data.shape

(671, 19569)

In [ ]:
# Check if there any null values in the BRCA_mRNA dataset
copy_number_variation_data.isnull().any().any()

False

In [ ]:
copy_number_variation_data.head(10)

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk,Label
TCGA.3C.AAAU.01,0.495252,-0.938762,0.750838,0.748593,1.093743,-0.876388,1.109460,-1.966983,-1.676420,1.090124,...,2.020624,1.972719,1.053381,1.254989,1.267331,0.963246,-0.580418,1.715811,-0.864183,1
TCGA.3C.AALI.01,1.028904,-1.575977,0.010864,0.005894,-1.575680,-0.967673,1.481901,-0.040373,-0.027748,1.213108,...,4.002085,3.897638,1.589442,-1.781406,-1.793233,1.446230,-0.790000,-1.268621,0.345052,4
TCGA.3C.AALJ.01,0.003059,0.405308,0.103201,0.098571,-0.706498,-1.643561,-0.661117,-1.342185,1.469986,-0.571499,...,-0.319977,-0.301093,-0.477515,-0.867769,-0.872324,0.039191,-1.421040,-0.811368,-1.357237,2
TCGA.3C.AALK.01,-0.258705,0.096910,-0.169130,-0.174763,0.199712,0.570857,-0.422884,-0.156210,-0.017618,-0.402285,...,-0.316675,-0.297884,-0.267586,0.055935,0.058733,-0.257130,-1.377441,0.098802,0.042601,1
TCGA.5L.AAT0.01,-0.083232,0.060782,-2.090765,-2.103474,0.376570,0.631333,-0.350660,-0.101592,0.100904,-0.312965,...,-0.307593,-0.289062,-0.200167,0.238663,0.242915,-0.185487,-0.999581,0.204322,0.078553,1
TCGA.5T.A9QA.01,-0.486242,2.050968,-0.216362,-0.222169,-2.334753,2.749902,-0.173333,-0.175416,0.011759,-0.110984,...,-0.399235,-0.392526,-0.141518,-2.457950,-2.475162,-1.970238,1.646581,-1.930651,-6.994250,2
TCGA.A1.A0SB.01,-0.111192,0.154506,-0.062751,-0.067992,0.400443,0.620683,-0.273585,0.005842,0.084190,-0.226338,...,-0.179623,-0.164744,-0.084514,0.258798,0.263210,-0.112381,0.848410,0.307432,0.189832,0
TCGA.A1.A0SE.01,-0.102997,0.018894,-0.056368,-0.061585,0.225047,0.497449,-0.151234,0.145686,-1.943348,-0.146892,...,-0.121831,-0.108601,0.039909,0.082111,0.085117,-0.106533,-0.727278,0.230822,0.289698,1
TCGA.A1.A0SF.01,0.687116,1.131535,0.857217,0.855364,1.226264,0.387146,-0.508045,1.277645,-1.716941,-0.444027,...,-0.210171,-0.194420,-0.356381,1.164884,1.176508,0.704939,-0.501251,-1.278740,-0.088652,1
TCGA.A1.A0SG.01,-0.213873,0.145081,-0.016369,-0.021439,0.390699,-0.143066,-0.384077,-0.019366,0.046708,-0.340793,...,-0.254754,-0.237731,-0.196879,0.203426,0.207398,-0.262491,0.866003,0.290087,0.104803,1


In [ ]:
print(copy_number_variation_data["Label"].value_counts()[0])
print(copy_number_variation_data["Label"].value_counts()[1])
print(copy_number_variation_data["Label"].value_counts()[2])
print(copy_number_variation_data["Label"].value_counts()[3])
print(copy_number_variation_data["Label"].value_counts()[4])

31
353
132
113
42


## BRCA_miRNA

In [ ]:
# Before transpose
miRNA_data.shape

(368, 671)

In [ ]:
# Transpose mrna_data
miRNA_data = miRNA_data.transpose()

In [ ]:
# After transpose
miRNA_data.head(10)

,hsa.let.7a.1,hsa.let.7a.2,hsa.let.7a.3,hsa.let.7b,hsa.let.7c,hsa.let.7d,hsa.let.7e,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,...,hsa.mir.935,hsa.mir.937,hsa.mir.939,hsa.mir.940,hsa.mir.942,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b
TCGA.3C.AAAU.01,0.068317,0.068932,0.073899,0.524562,-1.656853,-0.038283,0.501125,-2.390084,-2.406331,-0.760042,...,1.795470,0.108421,0.179279,1.768897,0.226659,0.943346,1.552613,-0.143572,-1.251868,1.603022
TCGA.3C.AALI.01,-0.301684,-0.318009,-0.301310,0.419859,-0.715963,0.460975,-1.999304,-0.659788,-0.651943,-1.050266,...,-0.409131,0.569264,0.188875,-0.453994,0.804807,-0.828843,0.599794,1.103662,-0.627587,-1.506825
TCGA.3C.AALJ.01,-0.150810,-0.122747,-0.126333,-0.958939,-0.971038,0.866585,2.074809,1.080746,1.088164,0.523204,...,0.742000,1.952380,-1.226285,-0.281690,-0.314543,-0.487207,1.261482,1.940594,-0.848552,0.787948
TCGA.3C.AALK.01,0.107831,0.097594,0.095545,0.615389,0.711952,-0.454282,0.227441,-0.735552,-0.686623,-1.015715,...,-0.715237,0.119549,-0.340371,-0.586990,-0.364370,-0.539524,0.856983,-0.831098,0.936474,0.030739
TCGA.5L.AAT0.01,0.395211,0.412879,0.418441,0.500594,0.426323,-1.545556,-0.952282,-0.280966,-0.233746,-0.818574,...,-0.925821,-0.248950,-1.439120,-0.707823,-0.109361,-0.490655,1.188748,-0.885905,0.449284,-0.645458
TCGA.5T.A9QA.01,1.135212,1.140183,1.143605,1.159087,-1.058870,0.348033,1.610267,-0.141723,-0.152147,1.243074,...,2.168396,-0.531098,-0.001485,-0.042380,2.139282,1.716711,0.375870,-0.922881,-1.316179,0.931785
TCGA.A1.A0SB.01,1.813887,1.790792,1.822272,2.113821,2.135373,1.844765,-0.469603,0.837358,0.798862,2.111705,...,-1.035240,-1.000344,-0.912012,-0.327511,0.306851,-0.311971,0.297224,0.390134,2.106744,0.319827
TCGA.A1.A0SE.01,0.473642,0.442224,0.431312,0.812876,1.031809,-0.453824,-0.611333,0.169462,0.184119,-0.511843,...,0.092706,-1.280723,-0.332270,-0.426424,-0.629303,-0.207033,0.066368,-0.330246,1.078635,0.089918
TCGA.A1.A0SF.01,-0.470612,-0.461217,-0.470734,-0.129307,0.864211,-1.070594,-0.571769,-0.000266,0.041877,-1.315689,...,-0.785645,-1.139447,-0.259122,-0.505870,-0.260053,-0.411471,0.268172,-0.481707,1.255710,-0.443763
TCGA.A1.A0SG.01,0.791968,0.841821,0.805374,0.867411,0.071249,0.499824,0.505744,0.859450,0.851887,0.162780,...,0.075832,1.512383,-1.105097,0.315259,-0.420010,1.331634,0.155349,0.490018,-0.062945,1.220345


In [ ]:
miRNA_data.shape

(671, 368)

In [ ]:
# Ensure the number of rows in data match with label
if miRNA_data.shape[0] == brca_Label.shape[0]:
    # Add the new column to the transposed DataFrame
    miRNA_data['Label'] = brca_Label['Label'].values
else:
    raise ValueError("The number of rows in must match the number of columns in the transposed df1.")

In [ ]:
miRNA_data.shape

(671, 369)

In [ ]:
# Check if there any null values in the BRCA_mRNA dataset
miRNA_data.isnull().any().any()

False

In [ ]:
miRNA_data.head(10)

,hsa.let.7a.1,hsa.let.7a.2,hsa.let.7a.3,hsa.let.7b,hsa.let.7c,hsa.let.7d,hsa.let.7e,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,...,hsa.mir.937,hsa.mir.939,hsa.mir.940,hsa.mir.942,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b,Label
TCGA.3C.AAAU.01,0.068317,0.068932,0.073899,0.524562,-1.656853,-0.038283,0.501125,-2.390084,-2.406331,-0.760042,...,0.108421,0.179279,1.768897,0.226659,0.943346,1.552613,-0.143572,-1.251868,1.603022,1
TCGA.3C.AALI.01,-0.301684,-0.318009,-0.301310,0.419859,-0.715963,0.460975,-1.999304,-0.659788,-0.651943,-1.050266,...,0.569264,0.188875,-0.453994,0.804807,-0.828843,0.599794,1.103662,-0.627587,-1.506825,4
TCGA.3C.AALJ.01,-0.150810,-0.122747,-0.126333,-0.958939,-0.971038,0.866585,2.074809,1.080746,1.088164,0.523204,...,1.952380,-1.226285,-0.281690,-0.314543,-0.487207,1.261482,1.940594,-0.848552,0.787948,2
TCGA.3C.AALK.01,0.107831,0.097594,0.095545,0.615389,0.711952,-0.454282,0.227441,-0.735552,-0.686623,-1.015715,...,0.119549,-0.340371,-0.586990,-0.364370,-0.539524,0.856983,-0.831098,0.936474,0.030739,1
TCGA.5L.AAT0.01,0.395211,0.412879,0.418441,0.500594,0.426323,-1.545556,-0.952282,-0.280966,-0.233746,-0.818574,...,-0.248950,-1.439120,-0.707823,-0.109361,-0.490655,1.188748,-0.885905,0.449284,-0.645458,1
TCGA.5T.A9QA.01,1.135212,1.140183,1.143605,1.159087,-1.058870,0.348033,1.610267,-0.141723,-0.152147,1.243074,...,-0.531098,-0.001485,-0.042380,2.139282,1.716711,0.375870,-0.922881,-1.316179,0.931785,2
TCGA.A1.A0SB.01,1.813887,1.790792,1.822272,2.113821,2.135373,1.844765,-0.469603,0.837358,0.798862,2.111705,...,-1.000344,-0.912012,-0.327511,0.306851,-0.311971,0.297224,0.390134,2.106744,0.319827,0
TCGA.A1.A0SE.01,0.473642,0.442224,0.431312,0.812876,1.031809,-0.453824,-0.611333,0.169462,0.184119,-0.511843,...,-1.280723,-0.332270,-0.426424,-0.629303,-0.207033,0.066368,-0.330246,1.078635,0.089918,1
TCGA.A1.A0SF.01,-0.470612,-0.461217,-0.470734,-0.129307,0.864211,-1.070594,-0.571769,-0.000266,0.041877,-1.315689,...,-1.139447,-0.259122,-0.505870,-0.260053,-0.411471,0.268172,-0.481707,1.255710,-0.443763,1
TCGA.A1.A0SG.01,0.791968,0.841821,0.805374,0.867411,0.071249,0.499824,0.505744,0.859450,0.851887,0.162780,...,1.512383,-1.105097,0.315259,-0.420010,1.331634,0.155349,0.490018,-0.062945,1.220345,1


In [ ]:
print(miRNA_data["Label"].value_counts()[0])
print(miRNA_data["Label"].value_counts()[1])
print(miRNA_data["Label"].value_counts()[2])
print(miRNA_data["Label"].value_counts()[3])
print(miRNA_data["Label"].value_counts()[4])

31
353
132
113
42


## BRCA_DNA_Methylation

In [ ]:
# Before transpose
DNA_Methylation_data.shape

(19049, 671)

In [ ]:
# Transpose mrna_data
DNA_Methylation_data = DNA_Methylation_data.transpose()

In [ ]:
# After transpose
DNA_Methylation_data.head(10)

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,-1.211396,-1.801239,-0.696763,0.832338,0.857098,-0.695053,-0.258891,0.601832,-0.100803,-5.232067,...,0.835761,-2.172061,-0.184735,0.821827,-0.842599,1.079773,0.780058,0.747015,0.378339,0.032275
TCGA.3C.AALI.01,0.974531,-1.091976,-1.597015,0.758754,0.450506,1.570301,0.330579,-0.912745,-0.682832,-1.232277,...,0.664052,-0.897618,-1.043726,0.746737,-0.188174,1.107008,0.404568,0.107306,0.271888,-0.147645
TCGA.3C.AALJ.01,0.633283,-0.258825,-0.873714,0.753608,0.821330,-0.369450,-0.315881,0.098281,-5.269406,-2.441139,...,-0.092097,-0.403899,0.163509,0.991162,-0.337345,1.143368,-0.583163,0.034379,-0.170242,0.748945
TCGA.3C.AALK.01,0.354642,0.344929,-1.080573,0.548382,0.497375,1.342464,-0.518794,-0.274796,-0.187178,-0.232070,...,0.277971,0.752473,0.722295,0.352692,-0.606283,1.327138,0.047150,0.014622,0.484366,0.173110
TCGA.5L.AAT0.01,-1.939963,-0.018208,-0.788607,0.589277,0.400632,1.553188,-0.299152,0.391160,-0.410440,-1.676137,...,-0.008991,0.693802,0.211610,0.025049,-0.391619,0.825476,0.272223,0.298979,0.029261,-0.424061
TCGA.5T.A9QA.01,1.328608,-1.007907,-0.583001,0.870132,1.183863,-0.876086,0.562873,3.940350,-0.422690,-1.472467,...,0.351988,1.575007,1.176298,0.677674,5.591173,1.014806,1.304881,1.151613,0.970602,2.038508
TCGA.A1.A0SB.01,0.946010,1.246215,-0.654375,-1.878898,-3.316770,-1.168542,0.646204,-0.857682,-0.588374,0.997476,...,-0.268241,0.144638,0.958181,-0.756234,-1.033358,-0.726589,-0.084193,-0.621557,-0.468274,-0.932571
TCGA.A1.A0SE.01,0.930281,0.360814,0.302439,0.160075,0.227579,1.418889,0.812077,0.537300,0.343098,0.563539,...,-0.279765,0.158648,-0.079927,-0.148292,-0.781399,0.116715,-0.078364,-0.887355,-0.221699,0.062171
TCGA.A1.A0SF.01,-0.519615,-0.426866,-1.561816,0.066117,0.051009,-0.012446,-0.205157,-0.532170,0.397713,0.714475,...,-0.642373,-0.263347,-0.086925,-0.473521,-0.112660,-0.967701,-0.585443,-0.625655,-0.738370,-0.914940
TCGA.A1.A0SG.01,0.554409,-1.357345,-0.836623,-0.338768,0.300039,-0.980884,-2.506433,-0.666725,0.234600,0.571410,...,-0.883780,0.150851,-0.688724,-0.388183,-0.283091,-0.634965,-0.713563,0.094450,-0.381204,0.623692


In [ ]:
DNA_Methylation_data.shape

(671, 19049)

In [ ]:
# Ensure the number of rows in data match with label
if DNA_Methylation_data.shape[0] == brca_Label.shape[0]:
    # Add the new column to the transposed DataFrame
    DNA_Methylation_data['Label'] = brca_Label['Label'].values
else:
    raise ValueError("The number of rows in must match the number of columns in the transposed df1.")

In [ ]:
DNA_Methylation_data.shape

(671, 19050)

In [ ]:
# Check if there any null values in the BRCA_mRNA dataset
DNA_Methylation_data.isnull().any().any()

False

In [ ]:
DNA_Methylation_data.head(10)

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk,Label
TCGA.3C.AAAU.01,-1.211396,-1.801239,-0.696763,0.832338,0.857098,-0.695053,-0.258891,0.601832,-0.100803,-5.232067,...,-2.172061,-0.184735,0.821827,-0.842599,1.079773,0.780058,0.747015,0.378339,0.032275,1
TCGA.3C.AALI.01,0.974531,-1.091976,-1.597015,0.758754,0.450506,1.570301,0.330579,-0.912745,-0.682832,-1.232277,...,-0.897618,-1.043726,0.746737,-0.188174,1.107008,0.404568,0.107306,0.271888,-0.147645,4
TCGA.3C.AALJ.01,0.633283,-0.258825,-0.873714,0.753608,0.821330,-0.369450,-0.315881,0.098281,-5.269406,-2.441139,...,-0.403899,0.163509,0.991162,-0.337345,1.143368,-0.583163,0.034379,-0.170242,0.748945,2
TCGA.3C.AALK.01,0.354642,0.344929,-1.080573,0.548382,0.497375,1.342464,-0.518794,-0.274796,-0.187178,-0.232070,...,0.752473,0.722295,0.352692,-0.606283,1.327138,0.047150,0.014622,0.484366,0.173110,1
TCGA.5L.AAT0.01,-1.939963,-0.018208,-0.788607,0.589277,0.400632,1.553188,-0.299152,0.391160,-0.410440,-1.676137,...,0.693802,0.211610,0.025049,-0.391619,0.825476,0.272223,0.298979,0.029261,-0.424061,1
TCGA.5T.A9QA.01,1.328608,-1.007907,-0.583001,0.870132,1.183863,-0.876086,0.562873,3.940350,-0.422690,-1.472467,...,1.575007,1.176298,0.677674,5.591173,1.014806,1.304881,1.151613,0.970602,2.038508,2
TCGA.A1.A0SB.01,0.946010,1.246215,-0.654375,-1.878898,-3.316770,-1.168542,0.646204,-0.857682,-0.588374,0.997476,...,0.144638,0.958181,-0.756234,-1.033358,-0.726589,-0.084193,-0.621557,-0.468274,-0.932571,0
TCGA.A1.A0SE.01,0.930281,0.360814,0.302439,0.160075,0.227579,1.418889,0.812077,0.537300,0.343098,0.563539,...,0.158648,-0.079927,-0.148292,-0.781399,0.116715,-0.078364,-0.887355,-0.221699,0.062171,1
TCGA.A1.A0SF.01,-0.519615,-0.426866,-1.561816,0.066117,0.051009,-0.012446,-0.205157,-0.532170,0.397713,0.714475,...,-0.263347,-0.086925,-0.473521,-0.112660,-0.967701,-0.585443,-0.625655,-0.738370,-0.914940,1
TCGA.A1.A0SG.01,0.554409,-1.357345,-0.836623,-0.338768,0.300039,-0.980884,-2.506433,-0.666725,0.234600,0.571410,...,0.150851,-0.688724,-0.388183,-0.283091,-0.634965,-0.713563,0.094450,-0.381204,0.623692,1


In [ ]:
print(DNA_Methylation_data["Label"].value_counts()[0])
print(DNA_Methylation_data["Label"].value_counts()[1])
print(DNA_Methylation_data["Label"].value_counts()[2])
print(DNA_Methylation_data["Label"].value_counts()[3])
print(DNA_Methylation_data["Label"].value_counts()[4])

31
353
132
113
42


## BRCA_mRNA

In [ ]:
# Before transpose
mrna_data.shape

(18206, 671)

In [ ]:
# Transpose mrna_data
mrna_data = mrna_data.transpose()

In [ ]:
# After transpose
mrna_data.head(10)

,A1BG,A1BG.AS1,A2M,A2M.AS1,A2ML1,A2MP1,A4GALT,AAAS,AACS,AADAT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA.3C.AAAU.01,1.389927,0.436758,-0.876187,1.607818,-0.415138,-0.374641,-1.727152,-0.562530,0.225676,-1.337786,...,0.382250,0.445598,2.359316,2.242234,1.190506,1.753777,2.013674,-0.424991,1.214399,1.667340
TCGA.3C.AALI.01,1.522872,1.815091,-0.508085,0.863019,-0.406608,-0.361230,-0.964244,0.525889,-0.590749,-0.724950,...,2.927253,0.455835,0.513900,0.179770,1.588651,1.298863,-0.140439,0.635518,0.302547,-1.162286
TCGA.3C.AALJ.01,1.788271,0.580557,-0.258138,-0.491954,-0.455969,0.855983,0.750127,-0.401461,1.077314,0.101880,...,-0.381820,1.900756,-0.505798,0.658038,-1.008467,2.105007,-0.334864,0.573943,-0.379244,-0.547104
TCGA.3C.AALK.01,-0.288543,-0.250194,-0.030804,-0.579638,-0.464938,-0.313092,0.480557,-0.426938,-0.739098,-0.091341,...,-0.029094,-0.210541,-0.185209,-0.066780,-0.759323,0.916528,-0.278901,0.515520,-0.939663,0.057166
TCGA.5L.AAT0.01,-0.462680,-0.091285,0.289970,0.572883,-0.438571,-0.090844,0.179004,0.685542,-0.623465,-0.421059,...,-0.232629,-0.709315,0.770811,1.161026,0.369677,-1.152811,0.075213,-0.100236,-0.485591,1.268702
TCGA.5T.A9QA.01,-0.451169,-0.688598,-1.641876,-0.836306,-0.479607,-1.014920,0.420826,0.697997,0.669477,1.067540,...,0.845089,1.473857,-0.869225,-0.460787,-0.008572,-1.553161,-1.397447,-0.472842,0.190737,-0.622851
TCGA.A1.A0SB.01,-0.805973,-1.037605,1.534552,0.099614,-0.444955,1.384198,0.260066,0.127438,1.339599,1.377738,...,0.051136,-1.227280,0.955570,-0.083160,1.287459,-1.086313,1.450555,1.176967,1.560267,0.954347
TCGA.A1.A0SE.01,-0.472368,0.658661,0.436717,-0.059346,-0.539629,-0.508012,0.058950,-0.117362,-0.443098,-0.566328,...,0.091780,0.231947,0.830896,0.605248,-0.282021,-0.380918,0.230984,-0.169919,-0.492343,0.581995
TCGA.A1.A0SF.01,0.112737,0.627872,0.476992,-0.092982,-0.408042,-0.264776,0.676890,1.583352,-1.331414,-1.010971,...,-1.173102,-0.059546,0.746254,0.158860,-0.112952,0.077010,0.050613,0.889745,-0.513877,-1.422932
TCGA.A1.A0SG.01,-0.397480,0.426477,0.147622,-0.666693,-0.457548,-0.343200,1.261371,0.208439,0.918643,-0.671109,...,-0.699790,-0.359577,-0.703121,-0.338345,0.205879,-1.682207,-0.605257,-0.606344,1.306838,-1.189883


In [ ]:
mrna_data.shape

(671, 18206)

In [ ]:
brca_Label.shape

(671, 1)

In [ ]:
# Ensure the number of rows in data match with label
if mrna_data.shape[0] == brca_Label.shape[0]:
    # Add the new column to the transposed DataFrame
    mrna_data['Label'] = brca_Label['Label'].values
else:
    raise ValueError("The number of rows in must match the number of columns in the transposed df1.")

In [ ]:
mrna_data.shape

(671, 18207)

In [ ]:
# Check if there any null values in the BRCA_mRNA dataset
mrna_data.isnull().any().any()

False

In [ ]:
mrna_data.head(10)

,A1BG,A1BG.AS1,A2M,A2M.AS1,A2ML1,A2MP1,A4GALT,AAAS,AACS,AADAT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Label
TCGA.3C.AAAU.01,1.389927,0.436758,-0.876187,1.607818,-0.415138,-0.374641,-1.727152,-0.562530,0.225676,-1.337786,...,0.445598,2.359316,2.242234,1.190506,1.753777,2.013674,-0.424991,1.214399,1.667340,1
TCGA.3C.AALI.01,1.522872,1.815091,-0.508085,0.863019,-0.406608,-0.361230,-0.964244,0.525889,-0.590749,-0.724950,...,0.455835,0.513900,0.179770,1.588651,1.298863,-0.140439,0.635518,0.302547,-1.162286,4
TCGA.3C.AALJ.01,1.788271,0.580557,-0.258138,-0.491954,-0.455969,0.855983,0.750127,-0.401461,1.077314,0.101880,...,1.900756,-0.505798,0.658038,-1.008467,2.105007,-0.334864,0.573943,-0.379244,-0.547104,2
TCGA.3C.AALK.01,-0.288543,-0.250194,-0.030804,-0.579638,-0.464938,-0.313092,0.480557,-0.426938,-0.739098,-0.091341,...,-0.210541,-0.185209,-0.066780,-0.759323,0.916528,-0.278901,0.515520,-0.939663,0.057166,1
TCGA.5L.AAT0.01,-0.462680,-0.091285,0.289970,0.572883,-0.438571,-0.090844,0.179004,0.685542,-0.623465,-0.421059,...,-0.709315,0.770811,1.161026,0.369677,-1.152811,0.075213,-0.100236,-0.485591,1.268702,1
TCGA.5T.A9QA.01,-0.451169,-0.688598,-1.641876,-0.836306,-0.479607,-1.014920,0.420826,0.697997,0.669477,1.067540,...,1.473857,-0.869225,-0.460787,-0.008572,-1.553161,-1.397447,-0.472842,0.190737,-0.622851,2
TCGA.A1.A0SB.01,-0.805973,-1.037605,1.534552,0.099614,-0.444955,1.384198,0.260066,0.127438,1.339599,1.377738,...,-1.227280,0.955570,-0.083160,1.287459,-1.086313,1.450555,1.176967,1.560267,0.954347,0
TCGA.A1.A0SE.01,-0.472368,0.658661,0.436717,-0.059346,-0.539629,-0.508012,0.058950,-0.117362,-0.443098,-0.566328,...,0.231947,0.830896,0.605248,-0.282021,-0.380918,0.230984,-0.169919,-0.492343,0.581995,1
TCGA.A1.A0SF.01,0.112737,0.627872,0.476992,-0.092982,-0.408042,-0.264776,0.676890,1.583352,-1.331414,-1.010971,...,-0.059546,0.746254,0.158860,-0.112952,0.077010,0.050613,0.889745,-0.513877,-1.422932,1
TCGA.A1.A0SG.01,-0.397480,0.426477,0.147622,-0.666693,-0.457548,-0.343200,1.261371,0.208439,0.918643,-0.671109,...,-0.359577,-0.703121,-0.338345,0.205879,-1.682207,-0.605257,-0.606344,1.306838,-1.189883,1


In [ ]:
print(mrna_data["Label"].value_counts()[0])
print(mrna_data["Label"].value_counts()[1])
print(mrna_data["Label"].value_counts()[2])
print(mrna_data["Label"].value_counts()[3])
print(mrna_data["Label"].value_counts()[4])

31
353
132
113
42


## BRCA_Multi_Omics

In [ ]:
# Before transpose
multi_omics_data.shape

(57191, 671)

In [ ]:
# Transpose mrna_data
multi_omics_data = multi_omics_data.transpose()

In [ ]:
# After transpose
multi_omics_data.head(10)

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk
TCGA.3C.AAAU.01,0.495252,-0.938762,0.750838,0.748593,1.093743,-0.876388,1.109460,-1.966983,-1.676420,1.090124,...,0.835761,-2.172061,-0.184735,0.821827,-0.842599,1.079773,0.780058,0.747015,0.378339,0.032275
TCGA.3C.AALI.01,1.028904,-1.575977,0.010864,0.005894,-1.575680,-0.967673,1.481901,-0.040373,-0.027748,1.213108,...,0.664052,-0.897618,-1.043726,0.746737,-0.188174,1.107008,0.404568,0.107306,0.271888,-0.147645
TCGA.3C.AALJ.01,0.003059,0.405308,0.103201,0.098571,-0.706498,-1.643561,-0.661117,-1.342185,1.469986,-0.571499,...,-0.092097,-0.403899,0.163509,0.991162,-0.337345,1.143368,-0.583163,0.034379,-0.170242,0.748945
TCGA.3C.AALK.01,-0.258705,0.096910,-0.169130,-0.174763,0.199712,0.570857,-0.422884,-0.156210,-0.017618,-0.402285,...,0.277971,0.752473,0.722295,0.352692,-0.606283,1.327138,0.047150,0.014622,0.484366,0.173110
TCGA.5L.AAT0.01,-0.083232,0.060782,-2.090765,-2.103474,0.376570,0.631333,-0.350660,-0.101592,0.100904,-0.312965,...,-0.008991,0.693802,0.211610,0.025049,-0.391619,0.825476,0.272223,0.298979,0.029261,-0.424061
TCGA.5T.A9QA.01,-0.486242,2.050968,-0.216362,-0.222169,-2.334753,2.749902,-0.173333,-0.175416,0.011759,-0.110984,...,0.351988,1.575007,1.176298,0.677674,5.591173,1.014806,1.304881,1.151613,0.970602,2.038508
TCGA.A1.A0SB.01,-0.111192,0.154506,-0.062751,-0.067992,0.400443,0.620683,-0.273585,0.005842,0.084190,-0.226338,...,-0.268241,0.144638,0.958181,-0.756234,-1.033358,-0.726589,-0.084193,-0.621557,-0.468274,-0.932571
TCGA.A1.A0SE.01,-0.102997,0.018894,-0.056368,-0.061585,0.225047,0.497449,-0.151234,0.145686,-1.943348,-0.146892,...,-0.279765,0.158648,-0.079927,-0.148292,-0.781399,0.116715,-0.078364,-0.887355,-0.221699,0.062171
TCGA.A1.A0SF.01,0.687116,1.131535,0.857217,0.855364,1.226264,0.387146,-0.508045,1.277645,-1.716941,-0.444027,...,-0.642373,-0.263347,-0.086925,-0.473521,-0.112660,-0.967701,-0.585443,-0.625655,-0.738370,-0.914940
TCGA.A1.A0SG.01,-0.213873,0.145081,-0.016369,-0.021439,0.390699,-0.143066,-0.384077,-0.019366,0.046708,-0.340793,...,-0.883780,0.150851,-0.688724,-0.388183,-0.283091,-0.634965,-0.713563,0.094450,-0.381204,0.623692


In [ ]:
multi_omics_data.shape

(671, 57191)

In [ ]:
brca_Label.shape

(671, 1)

In [ ]:
# Ensure the number of rows in data match with label
if multi_omics_data.shape[0] == brca_Label.shape[0]:
    # Add the new column to the transposed DataFrame
    multi_omics_data['Label'] = brca_Label['Label'].values
else:
    raise ValueError("The number of rows in must match the number of columns in the transposed df1.")

In [ ]:
multi_omics_data.shape

(671, 57192)

In [ ]:
# Check if there any null values in the BRCA_mRNA dataset
multi_omics_data.isnull().any().any()

False

In [ ]:
multi_omics_data.head(10)

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,pk,Label
TCGA.3C.AAAU.01,0.495252,-0.938762,0.750838,0.748593,1.093743,-0.876388,1.109460,-1.966983,-1.676420,1.090124,...,-2.172061,-0.184735,0.821827,-0.842599,1.079773,0.780058,0.747015,0.378339,0.032275,1
TCGA.3C.AALI.01,1.028904,-1.575977,0.010864,0.005894,-1.575680,-0.967673,1.481901,-0.040373,-0.027748,1.213108,...,-0.897618,-1.043726,0.746737,-0.188174,1.107008,0.404568,0.107306,0.271888,-0.147645,4
TCGA.3C.AALJ.01,0.003059,0.405308,0.103201,0.098571,-0.706498,-1.643561,-0.661117,-1.342185,1.469986,-0.571499,...,-0.403899,0.163509,0.991162,-0.337345,1.143368,-0.583163,0.034379,-0.170242,0.748945,2
TCGA.3C.AALK.01,-0.258705,0.096910,-0.169130,-0.174763,0.199712,0.570857,-0.422884,-0.156210,-0.017618,-0.402285,...,0.752473,0.722295,0.352692,-0.606283,1.327138,0.047150,0.014622,0.484366,0.173110,1
TCGA.5L.AAT0.01,-0.083232,0.060782,-2.090765,-2.103474,0.376570,0.631333,-0.350660,-0.101592,0.100904,-0.312965,...,0.693802,0.211610,0.025049,-0.391619,0.825476,0.272223,0.298979,0.029261,-0.424061,1
TCGA.5T.A9QA.01,-0.486242,2.050968,-0.216362,-0.222169,-2.334753,2.749902,-0.173333,-0.175416,0.011759,-0.110984,...,1.575007,1.176298,0.677674,5.591173,1.014806,1.304881,1.151613,0.970602,2.038508,2
TCGA.A1.A0SB.01,-0.111192,0.154506,-0.062751,-0.067992,0.400443,0.620683,-0.273585,0.005842,0.084190,-0.226338,...,0.144638,0.958181,-0.756234,-1.033358,-0.726589,-0.084193,-0.621557,-0.468274,-0.932571,0
TCGA.A1.A0SE.01,-0.102997,0.018894,-0.056368,-0.061585,0.225047,0.497449,-0.151234,0.145686,-1.943348,-0.146892,...,0.158648,-0.079927,-0.148292,-0.781399,0.116715,-0.078364,-0.887355,-0.221699,0.062171,1
TCGA.A1.A0SF.01,0.687116,1.131535,0.857217,0.855364,1.226264,0.387146,-0.508045,1.277645,-1.716941,-0.444027,...,-0.263347,-0.086925,-0.473521,-0.112660,-0.967701,-0.585443,-0.625655,-0.738370,-0.914940,1
TCGA.A1.A0SG.01,-0.213873,0.145081,-0.016369,-0.021439,0.390699,-0.143066,-0.384077,-0.019366,0.046708,-0.340793,...,0.150851,-0.688724,-0.388183,-0.283091,-0.634965,-0.713563,0.094450,-0.381204,0.623692,1


# Feature Selection

## BRCA_Copy Number Variation

In [ ]:
train_data, test_data = train_test_split(copy_number_variation_data,test_size=0.30,random_state=1)
target="Label"
X_train_cnv = train_data.select_dtypes(include=["number"]).copy()
y_train_cnv = train_data[target]
X_test_cnv = test_data.select_dtypes(include=["number"]).copy()
y_test_cnv = test_data[target]

In [ ]:
print("X train :", X_train_cnv.shape)
print("X test :", X_test_cnv.shape)
print("Y train :", y_train_cnv.shape)
print("Y test :", y_test_cnv.shape)

X train : (469, 19569)
X test : (202, 19569)
Y train : (469,)
Y test : (202,)


In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=19000, step=0.05)

# Fit RFE on the training data
rfe.fit(X_train_cnv, y_train_cnv)


KeyboardInterrupt: 

## BRCA_miRNA

###Training of SVM

In [ ]:
train_data, test_data = train_test_split(miRNA_data,test_size=0.30,random_state=1)
target="Label"
X_train_miRNA = train_data.select_dtypes(include=["number"]).copy()
y_train_miRNA = train_data[target]
X_test_miRNA = test_data.select_dtypes(include=["number"]).copy()
y_test_miRNA = test_data[target]

In [ ]:
print("X train :", X_train_miRNA.shape)
print("X test :", X_test_miRNA.shape)
print("Y train :", y_train_miRNA.shape)
print("Y test :", y_test_miRNA.shape)

X train : (469, 369)
X test : (202, 369)
Y train : (469,)
Y test : (202,)


In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=300, step=0.05)

# Fit RFE on the training data
rfe.fit(X_train_miRNA, y_train_miRNA)


KeyboardInterrupt: 

In [ ]:
# Transform the training and test sets to include only the selected features
x_train_rfe_miRNA = rfe.transform(X_train_miRNA)
x_test_rfe_miRNA = rfe.transform(X_test_miRNA)

In [ ]:
print(x_test_rfe_miRNA.shape)
print(x_train_rfe_miRNA.shape)


(202, 300)
(469, 300)


### Result

In [ ]:
# Convert the transformed arrays back to DataFrames
# Get the selected feature names
selected_features_miRNA = X_train_miRNA.columns[rfe.support_]

X_train_rfe_miRNA= pd.DataFrame(x_train_rfe_miRNA, columns=selected_features_miRNA, index=X_train_miRNA.index)
X_test_rfe_miRNA = pd.DataFrame(x_test_rfe_miRNA, columns=selected_features_miRNA, index=X_test_miRNA.index)

AttributeError: 'RFE' object has no attribute 'support_'

In [ ]:
# Save the dataframes
X_train_rfe_miRNA.to_pickle('X_train_rfe_miRNA.pkl')
X_test_rfe_miRNA.to_pickle('X_test_rfe_miRNA.pkl')

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')

X_train_rfe_miRNA.to_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_train_rfe_miRNA.pkl')
X_test_rfe_miRNA.to_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_test_rfe_miRNA.pkl')


## BRCA_DNA_Methylation

###Training of SVM

In [ ]:
train_data, test_data = train_test_split(DNA_Methylation_data,test_size=0.30,random_state=1)
target="Label"
X_train_dna = train_data.select_dtypes(include=["number"]).copy()
y_train_dna = train_data[target]
X_test_dna = test_data.select_dtypes(include=["number"]).copy()
y_test_dna = test_data[target]

In [ ]:
print("X train :", X_train_dna.shape)
print("X test :", X_test_dna.shape)
print("Y train :", y_train_dna.shape)
print("Y test :", y_test_dna.shape)

X train : (469, 19050)
X test : (202, 19050)
Y train : (469,)
Y test : (202,)


In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=19000, step=0.05)

# Fit RFE on the training data
rfe.fit(X_train_dna, y_train_dna)


RFE(estimator=SVC(kernel='linear'), n_features_to_select=19000)

In [ ]:
# Transform the training and test sets to include only the selected features
x_train_rfe_dna = rfe.transform(X_train_dna)
x_test_rfe_dna= rfe.transform(X_test_dna)

In [ ]:
print(x_test_rfe_dna.shape)
print(x_train_rfe_dna.shape)


(202, 19000)
(469, 19000)


### Result

In [ ]:
# Convert the transformed arrays back to DataFrames
# Get the selected feature names
selected_features_dna = X_train_dna.columns[rfe.support_]

X_train_rfe_dna = pd.DataFrame(x_train_rfe_dna, columns=selected_features_dna, index=X_train_dna.index)
X_test_rfe_dna = pd.DataFrame(x_test_rfe_dna, columns=selected_features_dna, index=X_test_dna.index)

In [ ]:
# Save the dataframes
X_train_rfe_dna.to_pickle('X_train_rfe_dna.pkl')
X_test_rfe_dna.to_pickle('X_test_rfe_dna.pkl')

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')

X_train_rfe_dna.to_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_train_rfe_dna.pkl')
X_test_rfe_dna.to_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_test_rfe_dna.pkl')


## BRCA_mRNA

###Training of SVM

In [ ]:
mrna_data

In [ ]:
train_data, test_data = train_test_split(mrna_data,test_size=0.30,random_state=1)
target="Label"
X_train_mrna = train_data.select_dtypes(include=["number"]).copy()
y_train_mrna = train_data[target]
X_test_mrna = test_data.select_dtypes(include=["number"]).copy()
y_test_mrna = test_data[target]

In [ ]:
print("X train :", X_train_mrna.shape)
print("X test :", X_test_mrna.shape)
print("Y train :", y_train_mrna.shape)
print("Y test :", y_test_mrna.shape)

X train : (469, 18207)
X test : (202, 18207)
Y train : (469,)
Y test : (202,)


In [ ]:
y_train_mrna

In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=18000, step=0.05)

# Fit RFE on the training data
rfe.fit(X_train_mrna, y_train_mrna)


In [ ]:
# Transform the training and test sets to include only the selected features
x_train_rfe_mrna = rfe.transform(X_train_mrna)
x_test_rfe_mrna = rfe.transform(X_test_mrna)

In [ ]:
print(x_test_rfe_mrna.shape)
print(x_train_rfe_mrna.shape)


### Result

In [ ]:
# Convert the transformed arrays back to DataFrames
# Get the selected feature names
selected_features_mrna = X_train_mrna.columns[rfe.support_]

X_train_rfe_mrna = pd.DataFrame(x_train_rfe_mrna, columns=selected_features_mrna, index=X_train_mrna.index)
X_test_rfe_mrna = pd.DataFrame(x_test_rfe_mrna, columns=selected_features_mrna, index=X_test_mrna.index)

In [ ]:
# Save the dataframes
X_train_rfe_mrna.to_pickle('X_train_rfe_mrna.pkl')
X_test_rfe_mrna.to_pickle('X_test_rfe_mrna.pkl')

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')

X_train_rfe_mrna.to_pickle('/content/gdrive/My Drive/BIOINFORMATICSII/Project/X_train_rfe_mrna.pkl')
X_test_rfe_mrna.to_pickle('/content/gdrive/My Drive/BIOINFORMATICSII/Project/X_test_rfe_mrna.pkl')


##BRCA_Multi_Omics


In [ ]:
train_data, test_data = train_test_split(multi_omics_data,test_size=0.30,random_state=1)
target="Label"
X_train_multi = train_data.select_dtypes(include=["number"]).copy()
y_train_multi = train_data[target]
X_test_multi = test_data.select_dtypes(include=["number"]).copy()
y_test_multi = test_data[target]

In [ ]:
print("X train :", X_train_multi.shape)
print("X test :", X_test_multi.shape)
print("Y train :", y_train_multi.shape)
print("Y test :", y_test_multi.shape)

X train : (469, 57192)
X test : (202, 57192)
Y train : (469,)
Y test : (202,)


In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=57000, step=0.05)

# Fit RFE on the training data
rfe.fit(X_train_multi, y_train_multi)


# Without SMOTE

In [ ]:
X_train_rfe_mrna = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_train_rfe_mrna.pkl')
X_test_rfe_mrna = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_test_rfe_mrna.pkl')


In [ ]:
### 1. Link notebook with google drive and access data from your personal Gdrive
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## BRCA_Copy Number Variation

In [ ]:
X_train_rfe_varNumber = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_train_rfe_copy_number_variation.pkl')
X_test_rfe_varNumber = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_test_rfe_copy_number_variation.pkl')


In [ ]:
X_test_rfe_varNumber.shape

(202, 19000)

In [ ]:
X_train_rfe_varNumber.shape

(469, 19000)

In [ ]:
y_train_cnv.shape

(469,)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
# Define the architecture parameters
input_dim = X_train_rfe_varNumber.shape[1]   # Number of features
hidden1_dim = 64
hidden2_dim = 32
encoding_dim = 16  # Size of the encoded representations

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Build the SDAE model
input_layer = Input(shape=(input_dim,))
hidden1 = Dense(hidden1_dim, activation='relu')(input_layer)
encoded1 = Dense(hidden2_dim, activation='relu')(hidden1)
encoded2 = Dense(encoding_dim, activation='relu')(encoded1)
decoded1 = Dense(hidden2_dim, activation='relu')(encoded2)
decoded2 = Dense(hidden1_dim, activation='relu')(decoded1)
output_layer = Dense(input_dim, activation='linear')(decoded2)

sdae = Model(inputs=input_layer, outputs=output_layer)

# Compile the SDAE model
sdae.compile(optimizer='adam', loss='mean_squared_error')

# Train the SDAE
sdae.fit(X_train_rfe_varNumber, X_train_rfe_varNumber, epochs=250, batch_size=32, shuffle=True, validation_data=(X_test_rfe_varNumber, X_test_rfe_varNumber), callbacks=[early_stopping])

# Extract the encoder part for feature extraction
encoder = Model(inputs=input_layer, outputs=encoded2)

# Extract features using the encoder
X_train_encoded = encoder.predict(X_train_rfe_varNumber)
X_test_encoded = encoder.predict(X_test_rfe_varNumber)

# Define a classifier (example: simple MLP classifier)
num_classes = len(np.unique(y_train_cnv))  # Number of classes in the target

classifier = Sequential([
    Dense(64, activation='relu', input_dim=encoding_dim),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multiclass classification
])


# Compile and train the classifier
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(X_train_encoded, y_train_cnv, epochs=250, batch_size=32, validation_data=(X_test_encoded, y_test_cnv), callbacks=[early_stopping])

# Predict probabilities
y_pred_prob = classifier.predict(X_test_encoded)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test_cnv, y_pred)
print(f"Accuracy: {accuracy}")

Epoch 1/250
15/15 [==============================] - 2s 67ms/step - loss: 1.0040 - val_loss: 0.9185
Epoch 2/250
15/15 [==============================] - 1s 63ms/step - loss: 0.9652 - val_loss: 0.8876
Epoch 3/250
15/15 [==============================] - 1s 73ms/step - loss: 0.9121 - val_loss: 0.8556
Epoch 4/250
15/15 [==============================] - 1s 65ms/step - loss: 0.8686 - val_loss: 0.8289
Epoch 5/250
15/15 [==============================] - 1s 62ms/step - loss: 0.8331 - val_loss: 0.8153
Epoch 6/250
15/15 [==============================] - 1s 69ms/step - loss: 0.8084 - val_loss: 0.8033
Epoch 7/250
15/15 [==============================] - 1s 44ms/step - loss: 0.7874 - val_loss: 0.7994
Epoch 8/250
15/15 [==============================] - 1s 43ms/step - loss: 0.7676 - val_loss: 0.7961
Epoch 9/250
15/15 [==============================] - 1s 43ms/step - loss: 0.7463 - val_loss: 0.7934
Epoch 10/250
15/15 [==============================] - 1s 43ms/step - loss: 0.7319 - val_loss: 0.7903

## BRCA_miRNA

In [ ]:
X_train_rfe_miRNA = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_train_rfe_miRNA.pkl')
X_test_rfe_miRNA = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_test_rfe_miRNA.pkl')

In [ ]:
X_train_rfe_miRNA.shape

(469, 300)

In [ ]:
print(y_train_miRNA.value_counts())

Label
1    245
2     96
3     82
4     25
0     21
Name: count, dtype: int64


In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
# Define the architecture parameters
input_dim = X_train_rfe_miRNA.shape[1]   # Number of features
hidden1_dim = 64
hidden2_dim = 32
encoding_dim = 16  # Size of the encoded representations

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Build the SDAE model
input_layer = Input(shape=(input_dim,))
hidden1 = Dense(hidden1_dim, activation='relu')(input_layer)
encoded1 = Dense(hidden2_dim, activation='relu')(hidden1)
encoded2 = Dense(encoding_dim, activation='relu')(encoded1)
decoded1 = Dense(hidden2_dim, activation='relu')(encoded2)
decoded2 = Dense(hidden1_dim, activation='relu')(decoded1)
output_layer = Dense(input_dim, activation='linear')(decoded2)

sdae = Model(inputs=input_layer, outputs=output_layer)

# Compile the SDAE model
sdae.compile(optimizer='adam', loss='mean_squared_error')

# Train the SDAE
sdae.fit(X_train_rfe_miRNA, X_train_rfe_miRNA, epochs=250, batch_size=32, shuffle=True, validation_data=(X_test_rfe_miRNA, X_test_rfe_miRNA), callbacks=[early_stopping])

# Extract the encoder part for feature extraction
encoder = Model(inputs=input_layer, outputs=encoded2)

# Extract features using the encoder
X_train_encoded = encoder.predict(X_train_rfe_miRNA)
X_test_encoded = encoder.predict(X_test_rfe_miRNA)

# Define a classifier (example: simple MLP classifier)
num_classes = len(np.unique(y_train_miRNA))  # Number of classes in the target

classifier = Sequential([
    Dense(64, activation='relu', input_dim=encoding_dim),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multiclass classification
])


# Compile and train the classifier
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(X_train_encoded, y_train_miRNA, epochs=250, batch_size=32, validation_data=(X_test_encoded, y_test_miRNA), callbacks=[early_stopping])

# Predict probabilities
y_pred_prob = classifier.predict(X_test_encoded)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test_miRNA, y_pred)
print(f"Accuracy: {accuracy}")

Epoch 1/250
15/15 [==============================] - 2s 23ms/step - loss: 0.9976 - val_loss: 1.0380
Epoch 2/250
15/15 [==============================] - 0s 6ms/step - loss: 0.9696 - val_loss: 0.9753
Epoch 3/250
15/15 [==============================] - 0s 6ms/step - loss: 0.8909 - val_loss: 0.8749
Epoch 4/250
15/15 [==============================] - 0s 5ms/step - loss: 0.8223 - val_loss: 0.8273
Epoch 5/250
15/15 [==============================] - 0s 6ms/step - loss: 0.7863 - val_loss: 0.7955
Epoch 6/250
15/15 [==============================] - 0s 6ms/step - loss: 0.7519 - val_loss: 0.7631
Epoch 7/250
15/15 [==============================] - 0s 6ms/step - loss: 0.7203 - val_loss: 0.7360
Epoch 8/250
15/15 [==============================] - 0s 5ms/step - loss: 0.6942 - val_loss: 0.7186
Epoch 9/250
15/15 [==============================] - 0s 5ms/step - loss: 0.6750 - val_loss: 0.7070
Epoch 10/250
15/15 [==============================] - 0s 5ms/step - loss: 0.6593 - val_loss: 0.6967
Epoch 11

## BRCA_DNA_Methylation

In [ ]:
X_train_rfe_dna = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_train_rfe_dna.pkl')
X_test_rfe_dna = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_test_rfe_dna.pkl')

In [ ]:
X_train_rfe_dna.shape

(469, 19000)

In [ ]:
print(y_train_dna.value_counts())

Label
1    245
2     96
3     82
4     25
0     21
Name: count, dtype: int64


In [ ]:
y_train_dna.shape

(469,)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
# Define the architecture parameters
input_dim = X_train_rfe_dna.shape[1]   # Number of features
hidden1_dim = 64
hidden2_dim = 32
encoding_dim = 16  # Size of the encoded representations

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Build the SDAE model
input_layer = Input(shape=(input_dim,))
hidden1 = Dense(hidden1_dim, activation='relu')(input_layer)
encoded1 = Dense(hidden2_dim, activation='relu')(hidden1)
encoded2 = Dense(encoding_dim, activation='relu')(encoded1)
decoded1 = Dense(hidden2_dim, activation='relu')(encoded2)
decoded2 = Dense(hidden1_dim, activation='relu')(decoded1)
output_layer = Dense(input_dim, activation='linear')(decoded2)

sdae = Model(inputs=input_layer, outputs=output_layer)

# Compile the SDAE model
sdae.compile(optimizer='adam', loss='mean_squared_error')

# Train the SDAE
sdae.fit(X_train_rfe_dna, X_train_rfe_dna, epochs=250, batch_size=32, shuffle=True, validation_data=(X_test_rfe_dna, X_test_rfe_dna), callbacks=[early_stopping])

# Extract the encoder part for feature extraction
encoder = Model(inputs=input_layer, outputs=encoded2)

# Extract features using the encoder
X_train_encoded = encoder.predict(X_train_rfe_dna)
X_test_encoded = encoder.predict(X_test_rfe_dna)

# Define a classifier (example: simple MLP classifier)
num_classes = len(np.unique(y_train_dna))  # Number of classes in the target

classifier = Sequential([
    Dense(64, activation='relu', input_dim=encoding_dim),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multiclass classification
])


# Compile and train the classifier
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(X_train_encoded, y_train_dna, epochs=250, batch_size=32, validation_data=(X_test_encoded, y_test_dna), callbacks=[early_stopping])

# Predict probabilities
y_pred_prob = classifier.predict(X_test_encoded)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test_dna, y_pred)
print(f"Accuracy: {accuracy}")

Epoch 1/250
15/15 [==============================] - 2s 66ms/step - loss: 0.9025 - val_loss: 0.7746
Epoch 2/250
15/15 [==============================] - 1s 44ms/step - loss: 0.7730 - val_loss: 0.7256
Epoch 3/250
15/15 [==============================] - 1s 47ms/step - loss: 0.7018 - val_loss: 0.6793
Epoch 4/250
15/15 [==============================] - 1s 48ms/step - loss: 0.6686 - val_loss: 0.6544
Epoch 5/250
15/15 [==============================] - 1s 42ms/step - loss: 0.6503 - val_loss: 0.6340
Epoch 6/250
15/15 [==============================] - 1s 44ms/step - loss: 0.6215 - val_loss: 0.6207
Epoch 7/250
15/15 [==============================] - 1s 42ms/step - loss: 0.6018 - val_loss: 0.6093
Epoch 8/250
15/15 [==============================] - 1s 43ms/step - loss: 0.5876 - val_loss: 0.6063
Epoch 9/250
15/15 [==============================] - 1s 43ms/step - loss: 0.5784 - val_loss: 0.6027
Epoch 10/250
15/15 [==============================] - 1s 42ms/step - loss: 0.5684 - val_loss: 0.5973

## BRCA_mRNA

In [ ]:
X_train_rfe_mrna.shape

(469, 18000)

In [ ]:
y_train_mrna.shape

(469,)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
# Define the architecture parameters
input_dim = X_train_rfe_mrna.shape[1]   # Number of features
hidden1_dim = 64
hidden2_dim = 32
encoding_dim = 16  # Size of the encoded representations

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Build the SDAE model
input_layer = Input(shape=(input_dim,))
hidden1 = Dense(hidden1_dim, activation='relu')(input_layer)
encoded1 = Dense(hidden2_dim, activation='relu')(hidden1)
encoded2 = Dense(encoding_dim, activation='relu')(encoded1)
decoded1 = Dense(hidden2_dim, activation='relu')(encoded2)
decoded2 = Dense(hidden1_dim, activation='relu')(decoded1)
output_layer = Dense(input_dim, activation='linear')(decoded2)

sdae = Model(inputs=input_layer, outputs=output_layer)

# Compile the SDAE model
sdae.compile(optimizer='adam', loss='mean_squared_error')

# Train the SDAE
sdae.fit(X_train_rfe_mrna, X_train_rfe_mrna, epochs=250, batch_size=32, shuffle=True, validation_data=(X_test_rfe_mrna, X_test_rfe_mrna), callbacks=[early_stopping])

# Extract the encoder part for feature extraction
encoder = Model(inputs=input_layer, outputs=encoded2)

# Extract features using the encoder
X_train_encoded = encoder.predict(X_train_rfe_mrna)
X_test_encoded = encoder.predict(X_test_rfe_mrna)

# Define a classifier (example: simple MLP classifier)
num_classes = len(np.unique(y_train_mrna))  # Number of classes in the target

classifier = Sequential([
    Dense(64, activation='relu', input_dim=encoding_dim),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multiclass classification
])


# Compile and train the classifier
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(X_train_encoded, y_train_mrna, epochs=250, batch_size=32, validation_data=(X_test_encoded, y_test_mrna), callbacks=[early_stopping])

# Predict probabilities
y_pred_prob = classifier.predict(X_test_encoded)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test_mrna, y_pred)
print(f"Accuracy: {accuracy}")

Epoch 1/250
15/15 [==============================] - 3s 91ms/step - loss: 0.9771 - val_loss: 0.9096
Epoch 2/250
15/15 [==============================] - 1s 70ms/step - loss: 0.8695 - val_loss: 0.8225
Epoch 3/250
15/15 [==============================] - 1s 42ms/step - loss: 0.8111 - val_loss: 0.7767
Epoch 4/250
15/15 [==============================] - 1s 44ms/step - loss: 0.7686 - val_loss: 0.7537
Epoch 5/250
15/15 [==============================] - 1s 39ms/step - loss: 0.7392 - val_loss: 0.7438
Epoch 6/250
15/15 [==============================] - 1s 40ms/step - loss: 0.7163 - val_loss: 0.7265
Epoch 7/250
15/15 [==============================] - 1s 40ms/step - loss: 0.6952 - val_loss: 0.7179
Epoch 8/250
15/15 [==============================] - 1s 40ms/step - loss: 0.6793 - val_loss: 0.7134
Epoch 9/250
15/15 [==============================] - 1s 45ms/step - loss: 0.6650 - val_loss: 0.7105
Epoch 10/250
15/15 [==============================] - 1s 41ms/step - loss: 0.6555 - val_loss: 0.7101

## Multi-Omics

In [ ]:
X_train_rfe_multiOmics = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_train_rfe_multi_omics.pkl')
X_test_rfe_multiOmics = pd.read_pickle('/content/gdrive/My Drive/BioinfoModelingAndSimulation/X_test_rfe_multi_omics.pkl')

In [ ]:
X_test_rfe_multiOmics.shape

(202, 57000)

In [ ]:
X_train_rfe_multiOmics.shape

(469, 57000)

In [ ]:
y_train_multi.shape

(469,)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
# Define the architecture parameters
input_dim = X_train_rfe_multiOmics.shape[1]   # Number of features
hidden1_dim = 64
hidden2_dim = 32
encoding_dim = 16  # Size of the encoded representations

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Build the SDAE model
input_layer = Input(shape=(input_dim,))
hidden1 = Dense(hidden1_dim, activation='relu')(input_layer)
encoded1 = Dense(hidden2_dim, activation='relu')(hidden1)
encoded2 = Dense(encoding_dim, activation='relu')(encoded1)
decoded1 = Dense(hidden2_dim, activation='relu')(encoded2)
decoded2 = Dense(hidden1_dim, activation='relu')(decoded1)
output_layer = Dense(input_dim, activation='linear')(decoded2)

sdae = Model(inputs=input_layer, outputs=output_layer)

# Compile the SDAE model
sdae.compile(optimizer='adam', loss='mean_squared_error')

# Train the SDAE
sdae.fit(X_train_rfe_multiOmics, X_train_rfe_multiOmics, epochs=250, batch_size=32, shuffle=True, validation_data=(X_test_rfe_multiOmics, X_test_rfe_multiOmics), callbacks=[early_stopping])

# Extract the encoder part for feature extraction
encoder = Model(inputs=input_layer, outputs=encoded2)

# Extract features using the encoder
X_train_encoded = encoder.predict(X_train_rfe_multiOmics)
X_test_encoded = encoder.predict(X_test_rfe_multiOmics)

# Define a classifier (example: simple MLP classifier)
num_classes = len(np.unique(y_train_multi))  # Number of classes in the target

classifier = Sequential([
    Dense(64, activation='relu', input_dim=encoding_dim),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multiclass classification
])


# Compile and train the classifier
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(X_train_encoded, y_train_multi, epochs=250, batch_size=32, validation_data=(X_test_encoded, y_test_multi), callbacks=[early_stopping])

# Predict probabilities
y_pred_prob = classifier.predict(X_test_encoded)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test_multi, y_pred)
print(f"Accuracy: {accuracy}")

Epoch 1/250
15/15 [==============================] - 4s 194ms/step - loss: 0.9917 - val_loss: 0.8774
Epoch 2/250
15/15 [==============================] - 2s 128ms/step - loss: 0.9056 - val_loss: 0.8535
Epoch 3/250
15/15 [==============================] - 2s 125ms/step - loss: 0.8765 - val_loss: 0.8328
Epoch 4/250
15/15 [==============================] - 2s 136ms/step - loss: 0.8439 - val_loss: 0.8031
Epoch 5/250
15/15 [==============================] - 2s 169ms/step - loss: 0.8136 - val_loss: 0.7971
Epoch 6/250
15/15 [==============================] - 3s 201ms/step - loss: 0.7888 - val_loss: 0.7871
Epoch 7/250
15/15 [==============================] - 2s 147ms/step - loss: 0.7738 - val_loss: 0.7835
Epoch 8/250
15/15 [==============================] - 2s 124ms/step - loss: 0.7582 - val_loss: 0.7820
Epoch 9/250
15/15 [==============================] - 2s 135ms/step - loss: 0.7476 - val_loss: 0.7810
Epoch 10/250
15/15 [==============================] - 2s 129ms/step - loss: 0.7451 - val_lo